In [138]:
from random import choice


In [256]:

# NOTE: make it so you don't go back to where you came from unless you have to  
    
# make a list of all edges - twice traverse gets in list twice 

def get_list_of_edges_by_traversal_count(edge_traversals_dict):
    """
    Given edge traversals dict, return a list of edges. 
    If the edge needs twice traversal it will appear twice in the 
    output list.

    """
    
    edges_list = []
    
    # look at each key (edge) in edge traversals dict
    for edge in edge_traversals_dict:
        
        # make var for number of traversals for current edge
        traversals = edge_traversals_dict[edge]
        
        # add the edge to the edges list as many times as it needs to be traversed
        for i in range(traversals):
        
            edges_list.append(edge)

    return edges_list


def get_route_order(start_node_, edge_traversals_dict_, nodes_dict_):
    
    node_order = []
    edges_traversed = []

    edges_list = get_list_of_edges_by_traversal_count(edge_traversals_dict_)
    
    print(edges_list)

    start_node = start_node_
    print(len(nodes_dict_[start_node_]))
    curr_node = start_node_ 
    
    previous_node = None
    
    while len(edges_list) > 0: # arbitrary while to start with

        print("\n\n----------------\nNOW AT:", curr_node)
        
        if len(node_order) > 0:
            previous_node = node_order[-1]
            
        print("previous_node:", previous_node)
        node_order.append(curr_node)
        print("nodes_order:", node_order)
        print("current_edges_list:", edges_list)

        print()
        # figure out where to go next
        # get all possible next nodes by looking at the connected edges

        possible_next_edges_all = list(nodes_dict_[curr_node]) # make set into list to support indexing
        
        
        # GET BRIDGES 
        
        # build a new graph 
        temp_G = nx.Graph()
        
        # add all untraversed edges to graph 
        # basically, you are considering edges already traversed as deleted 
        # node: dupe edges will only be added once 
        for edge in edges_list:
            edge_node_1, edge_node_2 = edge
            temp_G.add_edge(edge_node_1, edge_node_2)

        # get bridges from new graph
        bridges_all = list(nx.bridges(temp_G))
        
        # remove all edges that have more than one traversal remaining
        # from bridges list -- hacky way to acount for "parallel edges"
        for possible_bridge in bridges_all:
            
            num_traverses_remaining = edges_list.count(possible_bridge)
            
            if num_traverses_remaining > 1:
                bridges_all.remove(possible_bridge)
        
        
        print("possible_next_edges_all:", possible_next_edges_all)
        
        
        # find all possible next edges that are bridges
        # find all possible next edges that are NOT bridges 
            # then sort the non-bridges by other crteria 
            
        # turn pne_all and bridges_all into sets then find the intersection
        # these are the bridges 
        pne_bridges_set = set(bridges_all).intersection(set(possible_next_edges_all))

        # the non bridges that are possible next nodes are all possible next nodes
        # that are not bridges
        pne_non_bridge_set = set(possible_next_edges_all) - pne_bridges_set

        print("pne_bridges_set:", pne_bridges_set)
        print("pne_non_bridge_set:", pne_non_bridge_set)

        pne_bridge = list(pne_bridges_set)
        pne_non_bridge = list(pne_non_bridge_set)
        
        pne_no_prev_no_start = []
        pne_prev_no_start = []
        pne_prev_and_start = []
        pne_start_no_prev = []
        
        
        # get optimal order of non-bridges possible next edges 
        for edge in pne_non_bridge:
            print()
            inc_start_node = start_node in edge
            inc_prev_node = previous_node in edge
        #     print(f"{edge} includes {previous_node} (previous node)", (previous_node in edge))
        #     print(f"{edge} includes {start_node} (start_node)", (start_node in edge))


            if inc_start_node:
                # if edge includes start node check whether it also includes the previous node
                if inc_prev_node: 
                    # print(edge, f"inclues {previous_node} AND {start_node}")
                    # if it includes both add to prev and start list 
                    pne_prev_and_start.append(edge)
                else: 
                    # print(edge, f"does not inclue {previous_node} but does include {start_node}")
                    # if it only includes the start node, add to start list
                    pne_start_no_prev.append(edge)

            elif inc_prev_node: 
                # prev but not start
                # print(edge, f"inclues {previous_node} but not {start_node}")
                pne_prev_no_start.append(edge)

            else: 
                # print(edge, f"does not include {previous_node} or {start_node}")
                pne_no_prev_no_start.append(edge)


        possible_next_edges = pne_no_prev_no_start + pne_prev_no_start + pne_prev_and_start + pne_start_no_prev + pne_bridge
        
        print(possible_next_edges)

        keep_looking = True
        i = 0
        
        while keep_looking == True:
            
            print("\npossible_next_edges", possible_next_edges)

            print("current index, i:", i)

            # look through possible next edges for the current node 
            # if the edge is able to be traversed (still in edges_list)
            # make that edge the next edge_to_traverse and break out of loop
            # otherwise, try the next edge

            if possible_next_edges[i] in edges_list:

                edge_to_traverse = possible_next_edges[i]

                # remove the first instance of edge in list
                # (to account for edges that appear twice in list)
                edges_list.remove(edge_to_traverse)

                keep_looking = False


            # check if the reversed ordered edge is in the list 
            elif possible_next_edges[i][::-1] in edges_list:

                edge_to_traverse = possible_next_edges[i][::-1]

                # remove the first instance of edge in list
                # (to account for edges that appear twice in list)
                edges_list.remove(edge_to_traverse)

                keep_looking = False

            else: 
                if len(edges_list) > 0:
                    i = i + 1
                else:
                    keep_looking = False


        # make edge to traverse into list, so you can 
        # remove the current node and get a one-item list
        # then get that one item -- your next node

        print("next edge chosen:", edge_to_traverse)
        
        edge_to_traverse_lst = list(edge_to_traverse) # make edge to traverse tuple into a list
        
        edge_to_traverse_lst.remove(curr_node) # remove current node from edge to traverse list

        next_node = (edge_to_traverse_lst)[0] # next node is the only node left in the edge to traverse list

        edges_traversed.append(edge_to_traverse)
        print("edges_traversed:", edges_traversed)
        

        # make the next node your new curr node, and start the loop over
        curr_node = next_node
        print("next_node:", next_node)
    
    # add start node to the end of the node order, then print the final order
    node_order.append(start_node)
    print("\n\nFINAL NODE ORDER:", node_order)
    
    return node_order

In [254]:
# test graph 1
edge_traversals_dict_1 = {
      ('A','B') : 1,
      ('A','D') : 1,
      ('B','C') : 1,
      ('C','D') : 1,
}

nodes_dict_1 = {
    
    'A': { ('A','B'), ('A','D') },
    'B': { ('A','B'), ('B','C') },
    'C': { ('B','C'), ('C','D') },
    'D': { ('A','D'), ('C','D') }
}
   
# test graph 2   
edge_traversals_dict_2 = {
    ('A','B') : 2,
    ('A','D') : 1,
    ('B','C') : 1,
    ('C','D') : 2,
    ('E','A') : 1,
    ('E','B') : 1,
    ('E','C') : 1,
    ('E','D') : 1  
}
    
nodes_dict_2 = {
    'A': { ('A','B'), ('A','D'), ('A', 'E') },
    'B': { ('A','B'), ('B','C'), ('B', 'E') },
    'C': { ('B','C'), ('C','D'), ('C', 'E') },
    'D': { ('A','D'), ('C','D'), ('D', 'E') },
    'E': { ('A','E'),('B', 'E'), ('E','C'), ('E','D')}
}

# test 3 
edge_traversals_dict_3 = {
    ('A','B') : 1,
    ('A','D') : 2,
    ('B','C') : 1,
    ('C','D') : 1,
    ('E','A') : 1,
    ('E','B') : 1,
    ('E','C') : 1,
    ('E','D') : 1,
    ('F','B') : 1,
    ('F','C') : 1
}
    
nodes_dict_3 = {
    'A': { ('A','B'), ('A','D'), ('A', 'E')},
    'B': { ('A','B'), ('B','C'), ('B', 'E'), ('B','F')},
    'C': { ('B','C'), ('C','D'), ('C', 'E'), ('C','F')},
    'D': { ('A','D'), ('C','D'), ('D', 'E') },
    'E': { ('A','E'), ('B', 'E'), ('E','C'), ('E','D')},
    'F': { ('F','B'), ('F', 'C')}
}

# test 4 

edge_traversals_dict_4 = {
    ('A','B') : 1,
    ('A','H') : 1,
    ('A','G') : 2,
    
    ('B','C') : 1,
    
    ('C','D') : 2,
    ('C','H') : 1,
    
    ('D','E') : 2,
    
    ('E','H') : 1,
    ('E','F') : 1,
    
    ('F','G') : 1,
    
    ('G','H') : 1
}

nodes_dict_4 = {
    'A': { ('A','B'), ('A','H'), ('A','G') },
    
    'B': { ('A','B'), ('B','C') },
    
    'C': { ('B','C'), ('C','D'), ('C','H') },
    
    'D': { ('C','D'), ('D','E') },
    
    'E': { ('D','E'), ('E','H'), ('E','F') },
    
    'F': { ('E','F'), ('F','G') },
    
    'G': { ('A','G'), ('F','G'), ('G','H') },
    
    'H': { ('A','H'), ('C','H') , ('E','H'), ('G','H')}
}


nodes_dict_4['H']
# Tests 
# starts at start node
# ends at end node
# nodes list contains all nodes 
# test that all edges appear the correct number of times in the final edges traversed list?

{('A', 'H'), ('C', 'H'), ('E', 'H'), ('G', 'H')}

In [255]:
get_route_order('A', edge_traversals_dict_1, nodes_dict_1)
# get_route_order('A', edge_traversals_dict_2, nodes_dict_2)
# get_route_order('A', edge_traversals_dict_3, nodes_dict_3)
# get_route_order('A', edge_traversals_dict_4, nodes_dict_4)

[('A', 'B'), ('A', 'D'), ('B', 'C'), ('C', 'D')]
2


----------------
NOW AT: A
previous_node: None
nodes_order: ['A']
current_edges_list: [('A', 'B'), ('A', 'D'), ('B', 'C'), ('C', 'D')]

possible_next_edges_all: [('A', 'D'), ('A', 'B')]
pne_bridges_set: set()
pne_non_bridge_set: {('A', 'D'), ('A', 'B')}


[('A', 'D'), ('A', 'B')]

possible_next_edges [('A', 'D'), ('A', 'B')]
current index, i: 0
next edge chosen: ('A', 'D')
edges_traversed: [('A', 'D')]
next_node: D


----------------
NOW AT: D
previous_node: A
nodes_order: ['A', 'D']
current_edges_list: [('A', 'B'), ('B', 'C'), ('C', 'D')]

possible_next_edges_all: [('A', 'D'), ('C', 'D')]
pne_bridges_set: {('C', 'D')}
pne_non_bridge_set: {('A', 'D')}

[('A', 'D'), ('C', 'D')]

possible_next_edges [('A', 'D'), ('C', 'D')]
current index, i: 0

possible_next_edges [('A', 'D'), ('C', 'D')]
current index, i: 1
next edge chosen: ('C', 'D')
edges_traversed: [('A', 'D'), ('C', 'D')]
next_node: C


----------------
NOW AT: C
previous_node: D

['A', 'D', 'C', 'B', 'A']

In [231]:
# make graph from current edges list 
# get list of bridges in the graph
# if the "bridge" has more than one "traversal" left remove it from the bridges list and 
# treat like a non bridge 

import networkx as nx

temp_G = nx.Graph()

# small graph dict 1
edges_list = [
              ('A', 'B'), 
              ('A', 'B'), 
              ('A', 'D'), 
              ('B', 'C'), 
              ('C', 'D')
] 

# small graph dict 2
# edges_list = [('A', 'B'), ('A', 'B'), ('A', 'D'), ('B', 'C'), 
#               ('C', 'D'), ('C', 'D'), ('E', 'A'), ('E', 'B'), 
#               ('E', 'C'), ('E', 'D')]

# dict 4 

# edges_list = [('A', 'B'), ('A', 'H'), ('A', 'G'), ('A', 'G'), 
#               ('B', 'C'), ('C', 'D'), ('C', 'D'), ('C', 'H'), 
#               ('D', 'E'), ('D', 'E'), ('E', 'H'), ('E', 'F'), 
#               ('F', 'G'), ('G', 'H')]

# dict 4 - reduced 

# edges_list = [('A', 'B'), 
#               ('A', 'H'), 
#               ('A', 'G'), 
#               ('A', 'G'), 
#               ('B', 'C'), 
#               ('C', 'D'), ('C', 'D'), ('C', 'H'), 
#               ('D', 'E'), ('D', 'E'), ('E', 'H'), ('E', 'F'), 
#               ('F', 'G'), 
#               ('G', 'H')
#              ]

# add all edges in list (there are dupes) to temp_G
for edge in edges_list:
    edge_node_1, edge_node_2 = edge
    temp_G.add_edge(edge_node_1, edge_node_2)
    

# print(list(nx.bridges(temp_G)))


# so i need to have my own check for double traversals because they won't be counted twice


In [243]:
possible_next_edges_all = [
              ('A', 'B'), 
              ('A', 'B'), 
              ('A', 'D'), 
              ('B', 'C'), 
              ('C', 'D')] 

bridges_all = [('A', 'B'), 
              ('A', 'D'),
              ('X', 'Y')]


# pne_bridges = [('A', 'B'), ('A', 'D')] # INTERSECTION
# pne_non_bridge = [('B', 'C'), ('C', 'D')] ] # possible_ne_all = pne_bridges

# turn pne_all and bridges_all into sets then find the intersection
# these are the bridges 
pne_bridges_set = set(bridges_all).intersection(set(possible_next_edges_all))

pne_non_bridge_set = set(possible_next_edges_all) - pne_bridges_set

print("pne_bridges_set:", pne_bridges_set)
print("pne_non_bridge_set:", pne_non_bridge_set)



pne_bridges_set: {('A', 'D'), ('A', 'B')}
pne_non_bridge_set: {('B', 'C'), ('C', 'D')}
